# **Judea Pearl and the Ladder of Causation**

## The Ladder of Causation:

1. Rung 1: **Association** -> Observing -> "*How does observing X change my belief in Y?*";
2. Rung 2: **Intervention** -> Doing -> "*What will happen to Y if I do X?*";
3. Rung 3: **Counterfactual** -> Imagining -> "*If I had done X, what would Y be?*".